# Install dependencies on Ubuntu Linux
Users might need to install some dependencies on their system to be able to install all required R packages.
We provide here the list of dependencies needed on the Ubuntu Linux system.
Run the following command on Linux Shell to install the dependencies:

```bash
sudo apt-get -y install libcurl4-openssl-dev libxml2-dev libblas-dev liblapack-dev gfortran libpng-dev zlib1g-dev libudunits2-dev libssl-dev libgdal-dev
```

On some systems, the `preprocessCore` R package might encounter error with multiple threading.
If that case happens, we advise to install/reinstall the package with threading disable:

In [ ]:
if (!require("BiocManager", quietly = TRUE))
    install.packages("BiocManager")

BiocManager::install(c("preprocessCore", "oligo"), configure.args="--disable-threading", force = T, update = F)

## **Installation**

In [ ]:
# Install from CRAN
# install.packages("RCPA")

# Install from GitHub (developer's version)
install.packages("remotes")
remotes::install_github('tinnlab/RCPA', upgrade = F, force = T)

## **Load the required libraries**

In [ ]:
# Unload RCPA if being used
unloadNamespace("RCPA")
# Load the libraries:
library(RCPA)
library(SummarizedExperiment)
library(ggplot2)

## **Basic Protocol 1: PROCESSING AFFYMETRIX MICROARRAY**

Prepare the CELL files and sample information file:

In [ ]:
# Create a local directory to save the data:
userPath <- tempdir() # or replace tempdir() with user-defined directory path
downloadPath <- file.path(userPath, "GSE5281")
if(!dir.exists(downloadPath)) dir.create(downloadPath)

# Download the data
downloadedFiles <- RCPA::downloadGEO(GEOID = "GSE5281", 
                                platform = "GPL570", 
                                protocol = "affymetrix", 
                                destDir = downloadPath)

# Display downloadedFiles
head(downloadedFiles)

Read sample metadata

In [ ]:
affySampleInfo <- read.csv(file.path(downloadPath, "metadata.csv"))

# Display affyMetadata
head(affySampleInfo[, c("geo_accession", "characteristics_ch1.4", "characteristics_ch1.8")])

Process the CEL files

In [ ]:
# process downloaded Affymetrix data
affyExprs <- RCPA::processAffymetrix(dir = downloadPath, samples = affySampleInfo$geo_accession)

# Display affyExprs
head(affyExprs, c(5, 6))

Create `SummarizedExperiment` object for Affymemtrix dataset

In [ ]:
affyDataset <- SummarizedExperiment(assays = affyExprs, colData = affySampleInfo)
affyDataset

Access to the expression data stored in `SummarizedExperiment` object

In [ ]:
# access expression data
affyExprs <- SummarizedExperiment::assay(affyDataset)

# display affyExprs
head(affyExprs, c(5, 6))

Access to the sample information stored in `SummarizedExperiment` object

In [ ]:
# Access to sample information
affySampleInfo <- SummarizedExperiment::colData(affyDataset)

# display affySampleInfo
head(affySampleInfo[, c("title", "characteristics_ch1.4", "characteristics_ch1.8")])

## **Basic Prococol 2: PROCESSING AGILENT MICROARRAY**

Prepare the TXT files and sample information file

In [ ]:
# Create a local directory to save the downloaded data:
userPath <- tempdir()    # or user-defined directory path
downloadPath <- file.path(userPath, "GSE61196")
if(!dir.exists(downloadPath)) dir.create(downloadPath)

In [ ]:
# Download the data
downloadedFiles <- RCPA::downloadGEO(GEOID = "GSE61196", 
                                platform = "GPL4133", 
                                protocol = "agilent", 
                                destDir = downloadPath)

# Display downloaded files
head(downloadedFiles)

Read sample information and process Agilent TXT files

In [ ]:
agilSampleInfo <- read.csv(file.path(downloadPath, "metadata.csv"))

# Display affyMetadata
agilSampleInfo[9:14, c("geo_accession", "characteristics_ch2.1", "tissue.ch2")]

In [ ]:
# process downloaded Agilent data
agilExprs <- RCPA::processAgilent(dir = downloadPath, samples = agilSampleInfo$geo_accession, greenOnly = FALSE)
# Display agilExprs
agilExprs[9:14, 1:6]

Create a `SummarizedExperiment` object

In [ ]:
agilDataset <- SummarizedExperiment::SummarizedExperiment(assays = as.matrix(agilExprs), colData = agilSampleInfo)
agilDataset

Access to the experession data and sample information stored in `SummarizedExperiment` object

In [ ]:
# Access expression data
agilExprs <- SummarizedExperiment::assay(agilDataset)

# Display agilExprs
print(agilExprs[9:14, 1:6])

In [ ]:
# Access sample information
agilSampleInfo <- SummarizedExperiment::colData(agilDataset)

# Display agilSampleInfo
print(agilSampleInfo[9:14, c("geo_accession", "characteristics_ch2.1", "tissue.ch2")])

## **Support Protocol 1: PROCESSING RNA SEQUENCING (RNA-SEQ)**

First, we will get the sample information from GEO:

In [ ]:
userPath <- tempdir()    # or user-defined directory path
downloadPath <- file.path(userPath, "GSE153873")
if(!dir.exists(downloadPath)) dir.create(downloadPath)

# Download the GEO object to get metadata
GEOObject <- GEOquery::getGEO(GEO = "GSE153873", GSEMatrix = T, getGPL = T, destdir = downloadPath)

# Check the length of GEOObject
message("length: ", length(GEOObject))

# Extract the dataset from the GEOObject
samplesData <- GEOObject[[1]]

# Export sample data
metadata <- Biobase::pData(samplesData)

# save the metadata for later use
write.csv(metadata, file.path(downloadPath, "metadata.csv"))

# preview the metadata
as.data.frame(head(metadata[, c("title", "characteristics_ch1.1", "status")]))

Some datasets on GEO may be derived from different platforms. Therefore, the returned object is a list of different datasets. We can find out how many were used by checking the length of this object using `length()` function. Usually there will only be one platform and the dataset we want to analyse will be the first object in the list (`GEOObject[[1]]`).

Second, we will download the count matrix file

In [ ]:
# Download the supplementary files
GEOquery::getGEOSuppFiles(GEO = "GSE153873", fetch_files = TRUE, baseDir = userPath)

# Check the downloaded data files
list.files(file.path(userPath, "GSE153873"))

Examine the content of the count matrix file

In [ ]:
# read some lines of the count matrix file
countsFile <- file.path(userPath, "GSE153873", "GSE153873_summary_count.star.txt.gz")
lines <- readLines(countsFile, n = 10)

# display first 100 characters of each line
print(substr(lines, 1, 50))

Read sample information

In [ ]:
RNASeqSampleInfo <- read.csv(file.path(downloadPath, "metadata.csv"), row.names = 1)
head(RNASeqSampleInfo[, c("title", "characteristics_ch1.1", "status")])

In [ ]:
# read the count matrix file
countsData <- read.table(countsFile, header = TRUE, sep = "\t", fill = 0, row.names = 1, check.names = FALSE)
# Print out the first 5 rows and 6 columns
head(countsData, c(5,6))

# Rearrange the column of the count matrix
countsData <- countsData[, RNASeqSampleInfo$title]

# Rename the column names
colnames(countsData) <- rownames(RNASeqSampleInfo)

# Preview the count matrix
head(countsData, c(5,6))

In [ ]:
# Create the SummarizedExperiment object:
RNASeqDataset <- SummarizedExperiment::SummarizedExperiment(assays = as.matrix(countsData), colData = metadata)
print(RNASeqDataset)

# Access the expression data:
RNASeqExprs <- SummarizedExperiment::assay(RNASeqDataset)
# Display RNASeqExprs
message("RNASeqExprs:")
head(RNASeqExprs, c(5, 6))

# Access to the sample infomation data
RNASeqSampleInfo <- SummarizedExperiment::colData(RNASeqDataset)
# Display RNASeqSampleInfo
message("RNASeqSampleInfo:")
as.data.frame(RNASeqSampleInfo[1:5, c("characteristics_ch1", "characteristics_ch1.1", "disease state:ch1")])

## **Basic Protocol 3: DIFFERENTIAL ANALYSIS OF MICROARRAY DATA (AFFYMETRIX AND AGILENT)**

**Affymetrix dataset**

In [ ]:
# Load the Affymetrix data processed in Basic Protocol 1
affyDataset <- RCPA::loadData("affyDataset")

Now, we will create a design matrix for differential analysis:

In [ ]:
# Read metadata from the SummarizedExperiment object named affyDataset
affySampleInfo <- SummarizedExperiment::colData(affyDataset)
# Add a column specifying the condition of each sample (normal or Alzheimer’s)
affySampleInfo$condition <- ifelse(grepl("normal",
affySampleInfo$characteristics_ch1.8), "normal", "alzheimer")
# Factorize the new column
affySampleInfo$condition <- factor(affySampleInfo$condition)
# Add a new column to specify the region of the sample tissue,
# use make.names() to remove special characters and
# use tolower() to make all characters lower case
affySampleInfo$region <- make.names(affySampleInfo$characteristics_ch1.4)
affySampleInfo$region <- tolower(affySampleInfo$region)
# Factorize the newly added column
affySampleInfo$region <- factor(affySampleInfo$region)
# Update the affyDataset object
SummarizedExperiment::colData(affyDataset) <- affySampleInfo
# Create a design matrix
affyDesign <- model.matrix(~0 +  condition + region + condition:region, data = affySampleInfo)
# Remove special characters in column names
colnames(affyDesign) <- make.names(colnames(affyDesign))
# Print the design matrix
affyDesign[1:5, 1:3]

Next, we create a contrast matrix for differential analysis

In [ ]:
# Create a constrast matrix
affyContrast <- limma::makeContrasts(conditionalzheimer-conditionnormal, levels = affyDesign)
# Print contrast
head(affyContrast)

Finally, we can perform differential analysis as follows:

In [ ]:
# GPL570 requires package hgu133plus2.db to be installed
# You will be asked to install the package if it is not installed
# Here we install it beforehand
BiocManager::install("hgu133plus2.db", update = F)

# Run differential expression analysis
affyDEExperiment <- RCPA::runDEAnalysis(affyDataset, method = "limma", 
                                        design = affyDesign, contrast = affyContrast, 
                                        annotation = "GPL570")
# Display affyDEExperiment
affyDEExperiment

The function `runDEAnalysis()` also returns a `SummarizedExperiment` object, in which the differential analysis result is stored under `rowData` attribute. We can use the following code to access to the result:

In [ ]:
# Extract the differential analysis result
affyDEResults <- SummarizedExperiment::rowData(affyDEExperiment)
# Print in R console
as.data.frame(head(affyDEResults, c(3,5)))

We can visualize the differential analysis result using an MA plot:

In [ ]:
# This code is specific for Jupyter notebook environment to control the plot size
options(repr.plot.width = 10, repr.plot.height = 10)

In [ ]:
 # Visualize the differential analysis results using MA plot
RCPA::plotMA(affyDEResults, logFCThreshold = 0.5) + ggplot2::ggtitle("Affymetrix - GSE5281")

We can also use Volcano plot instead:

In [ ]:
# Visualize the differential analysis results using volcano plot
RCPA::plotVolcanoDE(affyDEResults, logFCThreshold = 0.5) + ggplot2::ggtitle("Affymetrix - GSE5281")

**Agilent dataset**

In [ ]:
# Load the Agilent data processed in Support Protocol 1
agilDataset <- RCPA::loadData("agilDataset")

We also start with creating design matrix and contrast matrix

In [ ]:

# Access to the sample infomation data
agilSampleInfo <- SummarizedExperiment::colData(agilDataset)

# Add a column specifying the condition of the sample,
# which can be either normal or alzheimer
agilSampleConditions <- ifelse(grepl("healthy", agilSampleInfo$source_name_ch1), "normal", "alzheimer")
# Factorize the newly added column
agilSampleInfo$condition <- factor(agilSampleConditions)
# Update the colData attribute with new column
SummarizedExperiment::colData(agilDataset) <- agilSampleInfo

# Create a design matrix
agilDesign <- model.matrix(~0 + condition, data = SummarizedExperiment::colData(agilDataset))
# Create a constrast matrix
agilContrast <- limma::makeContrasts("conditionalzheimer-conditionnormal", levels = agilDesign)

The platform of the dataset GSE61196 is GPL4133, whose annotation is not included in our package. Therefore, we need to manually create a mapping data frame that will be used as `annotation` input of `runDEAnalysis()` function. The following are the steps that users can use to obtain these information:

In [ ]:
# Download the information for GPL4133 platform:
GPL4133Pl <- GEOquery::getGEO(GEO = "GPL4133")

# Access to the dataTable slot in GPL4133Pl:
GPL4133Anno <- GEOquery::dataTable(GPL4133Pl)

# Access to annotation data
GPL4133AnnoTbl <- GEOquery::Table(GPL4133Anno)

# Display annotation (GPL4133AnnoTbl)
GPL4133AnnoTbl[9:14, c("SPOT_ID", "GENE", "GENE_SYMBOL")]

In [ ]:

# Create the mapping data frame
GPL4133GeneMapping <- data.frame(FROM = GPL4133AnnoTbl$SPOT_ID, 
                                TO = as.character(GPL4133AnnoTbl$GENE), 
                                stringsAsFactors = F)
                                
#Display GPL4133GeneMapping:
GPL4133GeneMapping[15:20,]

We then can perform differential analysis as follows:

In [ ]:
# Run differential expression analysis
agilDEExperiment <- RCPA::runDEAnalysis(agilDataset, method = "limma", 
                                        design = agilDesign, contrast = agilContrast, 
                                        annotation = GPL4133GeneMapping)

# Extract the outcome of differential expression analysis
agilDEResults <- SummarizedExperiment::rowData(agilDEExperiment)

# Print in R console
as.data.frame(head(agilDEResults, c(3,5)))

Results visualization using MA plot and Volcano plot:

In [ ]:
# MA plot
RCPA::plotMA(agilDEResults, logFCThreshold = 0.5) +
  ggplot2::ggtitle("Agilent - GSE61196")
      
# Volcano plot
RCPA::plotVolcanoDE(agilDEResults, logFCThreshold = 0.5) +
  ggplot2::ggtitle("Agilent - GSE61196")

## **Support Protocol 4: DIFFERENTIAL ANALYSIS OF RNA-SEQ DATA**

In [ ]:
# Load the RNA-Seq dataset
RNASeqDataset <- RCPA::loadData("RNASeqDataset")

Create the design and contrast matrices:

In [ ]:
# Access to the sample infomation data
RNASeqSampleInfo <- SummarizedExperiment::colData(RNASeqDataset)

# Add a column specifying the condition of the sample,
# which can be either normal or alzheimer
RNASeqSampleConditions <- ifelse(grepl("Alzheimer", RNASeqSampleInfo$characteristics_ch1.1), "alzheimer", "normal")

# Factorize the newly added column
RNASeqSampleInfo$condition <- factor(RNASeqSampleConditions)

# Update the colData attribute with new column
SummarizedExperiment::colData(RNASeqDataset) <- RNASeqSampleInfo

# Create design matrix
RNASeqDesign <- model.matrix(~0 + condition, data = SummarizedExperiment::colData(RNASeqDataset))

# Create constrast matrix
RNASeqContrast <- limma::makeContrasts("conditionalzheimer-conditionnormal", levels = RNASeqDesign)

Create a mapping between the ID used in the assay data and Entrez ID:

In [ ]:
# Install the genome wide annotation database for human
if (!require("BiocManager", quietly = TRUE)) {
    install.packages("BiocManager")
}
BiocManager::install("org.Hs.eg.db")

# Load the annotation database
library(org.Hs.eg.db)

# Get current gene IDs used in RNA-Seq dataset
RNASeqGeneIDs <- rownames(RNASeqDataset)

# Create a mapping dataframe
ENSEMBLMapping <- AnnotationDbi::select(x = org.Hs.eg.db, 
                                        keys = RNASeqGeneIDs, 
                                        keytype = "SYMBOL", 
                                        columns = c("SYMBOL", "ENTREZID"))  
colnames(ENSEMBLMapping) <- c("FROM", "TO")

# Print first 6 rows into R console
head(ENSEMBLMapping)

Perform differential analysis using the function `runDEAnalysis()`:

In [ ]:
# Perform differential analysis
RNASeqDEExperiment <- RCPA::runDEAnalysis(RNASeqDataset, method = "DESeq2", 
                                        design = RNASeqDesign, contrast = RNASeqContrast, 
                                        annotation = ENSEMBLMapping)

# Extract the differential analysis results
RNASeqDEResults <- SummarizedExperiment::rowData(RNASeqDEExperiment)
# Print out the obtained DE analysis results
as.data.frame(head(RNASeqDEResults, c(3,5)))

Visualize the results using MA and Volcano plot:

In [ ]:
# MA plot
RCPA::plotMA(RNASeqDEResults, logFCThreshold = 0.5) +
    ggplot2::ggtitle("RNASeq - GSE153873")
# Volcano plot
RCPA::plotVolcanoDE(RNASeqDEResults, logFCThreshold = 0.5) +
    ggplot2::ggtitle("RNASeq - GSE153873")

## **Basic Protocol 5: GENE SET ENRICHMENT ANALYSIS**

In [ ]:
# Loading differential results for Affymetrix data
affyDEExperiment <- RCPA::loadData("affyDEExperiment")
# Loading the results for Agilent data
agilDEExperiment <- RCPA::loadData("agilDEExperiment")
# Loading the results for RNA-Seq data
RNASeqDEExperiment <- RCPA::loadData("RNASeqDEExperiment")

We will need to prepare the gene sets using the function `getGeneSets`:

In [ ]:
# Download gene sets from KEGG for human
KEGGGenesets <- RCPA::getGeneSets(database = "KEGG", org = "hsa")

# OR Download the gene sets from GO database
# taxid = 9606 for human 
GOTerms <- RCPA::getGeneSets(database = "GO", taxid = 9606, 
                            namespace = "biological_process")

We will use the KEGG gene sets and perform enrichment analysis on them. Let us start with enrichment analysis using KS test:

In [ ]:
# Set seed to create reproducible results:
set.seed(1)

# Enrichment analysis using KS test and KEGG pathways:
# For Affymetrix dataset:
affyKSResult <- RCPA::runGeneSetAnalysis(summarizedExperiment = affyDEExperiment,
                                        genesets = KEGGGenesets, method = "ks")

# For Agilent dataset:
agilKSResult <- RCPA::runGeneSetAnalysis(summarizedExperiment = agilDEExperiment,
                                        genesets = KEGGGenesets, method = "ks")

# For RNA-Seq dataset:
RNASeqKSResult <- RCPA::runGeneSetAnalysis(summarizedExperiment = RNASeqDEExperiment,
                                        genesets = KEGGGenesets, method = "ks")


# Display the result for KS test, for example, using RNA-Seq dataset:
RNASeqKSResult[1:5, c("ID", "p.value", "pFDR", "score", "name")]

Enrichment analysis using Wilcoxon test:

In [ ]:
# Enrichment analysis using Wilcoxon test and KEGG pathways
set.seed(1)

# For Affymetrix dataset:
affyWilcoxResult <- RCPA::runGeneSetAnalysis(summarizedExperiment = affyDEExperiment, 
                                            genesets = KEGGGenesets, 
                                            method = "wilcox")

# For Agilent dataset:
agilWilcoxResult <- RCPA::runGeneSetAnalysis(summarizedExperiment = agilDEExperiment, 
                                            genesets = KEGGGenesets, 
                                            method = "wilcox")

# For RNA-Seq dataset:
RNASeqWilcoxResult <- RCPA::runGeneSetAnalysis(summarizedExperiment = RNASeqDEExperiment, 
                                            genesets = KEGGGenesets, 
                                            method = "wilcox")

# Display the result for Wilcoxon test, for example, using RNA-Seq dataset
RNASeqWilcoxResult[1:5, c("ID", "p.value", "pFDR", "score", "name")]

Perform enrichment analysis using over-representation analysis (ORA):

In [ ]:
# Specify the threshold to identify DE genes, which are required for ORA
oraArgsList <- list(pThreshold = 0.05)

# Set seed to create reproducible results
set.seed(1)

# Enrichment analysis using ORA and KEGG pathways
# For Affymetrix dataset:
affyORAResult <- RCPA::runGeneSetAnalysis(summarizedExperiment = affyDEExperiment,
                                          genesets = KEGGGenesets, method = "ora",
                                          ORAArgs = oraArgsList)

# For Agilent dataset:
agilORAResult <- RCPA::runGeneSetAnalysis(summarizedExperiment = agilDEExperiment,
                                          genesets = KEGGGenesets, method = "ora",
                                          ORAArgs = oraArgsList)

# For RNA-Seq dataset:
RNASeqORAResult <- RCPA::runGeneSetAnalysis(summarizedExperiment = RNASeqDEExperiment,
                                          genesets = KEGGGenesets, method = "ora",
                                          ORAArgs = oraArgsList)

# Display, for example, the result of RNA-Seq data:
RNASeqORAResult[1:5, c("ID", "p.value", "pFDR", "score", "name")]

Perform enrichment analysis using fast gene set enrichment analysis (FGSEA):

In [ ]:
# Running fgsea requires the package fgsea to be installed
# The runGeneSetAnalysis will ask to install the package if it is not already installed
# Here we install it beforehand
BiocManager::install("fgsea", update = F)

# Specify a list of arguments tailored for FGSEA:
FGSEAArgsList <- list(minSize = 10)

# Set seed to create reproducible results
set.seed(1)

# Enrichment analysis using FSGEA and KEGG pathway:
# For Affymetrix dataset:
affyFGSEAResult <- RCPA::runGeneSetAnalysis(summarizedExperiment = affyDEExperiment,
                                            genesets = KEGGGenesets, method = "fgsea", 
                                            FgseaArgs = FGSEAArgsList)

# For Agilent dataset:
agilFGSEAResult <- RCPA::runGeneSetAnalysis(summarizedExperiment = agilDEExperiment,
                                            genesets = KEGGGenesets, method = "fgsea", 
                                            FgseaArgs = FGSEAArgsList)

# For RNA-Seq dataset:
RNASeqFGSEAResult <- RCPA::runGeneSetAnalysis(summarizedExperiment = RNASeqDEExperiment,
                                            genesets = KEGGGenesets, method = "fgsea",
                                            FgseaArgs = FGSEAArgsList)

# Display the result of RNA-Seq using FGSEA, for example:
RNASeqFGSEAResult[c(1:5), c("ID", "p.value", "pFDR", "score", "name")]

Perform enrichment analysis using gene set analysis (GSA):

In [ ]:
# Running GSA requires the package GSA to be installed
# The runGeneSetAnalysis will ask to install the package if it is not already installed
# Here we install it beforehand
BiocManager::install("GSA", update = F)

# Specify the list of arguments customized for GSA
GSAArgsList <- list(method = "maxmean", minsize = 15, maxsize = 500, nperms = 1000)

# Set seed to create reproducible results
set.seed(1)

# Enrichment analysis using GSA and KEGG pathways
# For Affymetrix dataset:
affyGSAResult <- RCPA::runGeneSetAnalysis(summarizedExperiment = affyDEExperiment,
                                          genesets = KEGGGenesets, method = "gsa", 
                                          GSAArgs = GSAArgsList)

# For Agilent dataset:
agilGSAResult <- RCPA::runGeneSetAnalysis(summarizedExperiment = agilDEExperiment,
                                          genesets = KEGGGenesets, method = "gsa", 
                                          GSAArgs = GSAArgsList)

# For RNA-Seq dataset
RNASeqGSAResult <- RCPA::runGeneSetAnalysis(summarizedExperiment = RNASeqDEExperiment,
                                        genesets = KEGGGenesets, method = "gsa",
                                        GSAArgs = GSAArgsList)

# Display the result of RNA-Seq using GSA:
RNASeqGSAResult[c(1:5), c("ID", "p.value", "pFDR", "score", "name")]

Visualize the results using Volcano plot:

In [ ]:
# Visualize the enrichment result of RNA-Seq using FGSEA
# We can replace RNASeqFGSEAResult with other results as shown above:
RCPA::plotVolcanoPathway(PAResult = RNASeqFGSEAResult, topToLabel = 10) +
        ggplot2::ggtitle("RNASeq - GSE153873 - FGSEA")

Visualize the results using forest plot:

In [ ]:
# Create a list containing the top 20 pathways from the result
# We can replace RNASeqFGSEAResult by other results as shown above:
RNASeqFGSEAToPlot <- list("RNASeq - GSE153873 - FGSEA" = RNASeqFGSEAResult[1:20,])

# Generate forest plot:
RCPA::plotForest(resultsList = RNASeqFGSEAToPlot, yAxis = "name", statLims = c(-3.5, 1))

Visualize the enrichment analysis results as a network of pathway graph (the function in this snippet requires a web browser to run):

In [ ]:
# This code is specific for Jupyter Notebook to show the pathway network plots with a reasonable height
IRdisplay::display_javascript('
const style = document.createElement("style")
style.textContent = "iframe.jp-OutputArea-output { height: 500px; }" //change this to modify the height of the plot
document.head.appendChild(style)
')

In [ ]:
# Select top 20 pathways from the results
# We can replace RNASeqFGSEAResult with other results as shown above:
RNASeqFGSEAToPlot <- list("RNASeq - GSE153873 - FGSEA" = RNASeqFGSEAResult[1:20,])

# Get IDs for top 20 pathways
selectedPathways <- RNASeqFGSEAResult$ID[1:20]

# Generate network graph of selected pathways
pltHtml <- RCPA::plotPathwayNetwork(
    PAResults = RNASeqFGSEAToPlot,
    genesets = KEGGGenesets,
    selectedPathways = selectedPathways,
    statistic = "normalizedScore",
    mode = "continuous",
    edgeThreshold = 0.75
)

## **Basic Protocol 6: TOPOLOGY-BASED (TB) PATHWAY ANALYSIS**

In [ ]:
# loading differential results for Affymetrix data
affyDEExperiment <- RCPA::loadData("affyDEExperiment")
# loading the results for Agilent data
agilDEExperiment <- RCPA::loadData("agilDEExperiment")
# loading the results for RNA-Seq data
RNASeqDEExperiment <- RCPA::loadData("RNASeqDEExperiment")

Retrieve pathway topology from KEGG for SPIA:

In [ ]:
# Install required dependencies beforehand
BiocManager::install("ROntoTools", update = F)

# Retrieve gene networks from KEGG for SPIA
SPIANetwork <- RCPA::getSPIAKEGGNetwork(org = "hsa", updateCache = FALSE)

Run SPIA on the three datasets:

In [ ]:

# Specify the list of arguments specific for SPIA
SPIAArgsList <- list(nB = 1000, pThreshold = 0.05)

# Set seed to create reproducible results
set.seed(1)

# Run SPIA on Affymetrix dataset:
affySPIAResult <- RCPA::runPathwayAnalysis(summarizedExperiment = affyDEExperiment,
                                           network = SPIANetwork, method = "spia", 
                                           SPIAArgs = SPIAArgsList)

# Run SPIA on Agilent dataset:
agilSPIAResult <- RCPA::runPathwayAnalysis(summarizedExperiment = agilDEExperiment,
                                           network = SPIANetwork, method = "spia", 
                                           SPIAArgs = SPIAArgsList)

# Run SPIA on RNA-Seq dataset:
RNASeqSPIAResult <- RCPA::runPathwayAnalysis(summarizedExperiment = RNASeqDEExperiment,
                                            network = SPIANetwork, method = "spia",
                                            SPIAArgs = SPIAArgsList)

# Display the result for RNA-Seq dataset:
RNASeqSPIAResult[1:5, c("ID", "p.value", "pFDR", "score", "name")]

Retreive pathway topology from KEGG for CePa ORA and CePa GSA:

In [ ]:
# Install required dependencies beforehand
BiocManager::install("CePa", update = F)

# Retrieve pathway information from KEGG for CePa ORA and CePa GSA:
CePaNetwork <- RCPA::getCePaPathwayCatalogue(org = "hsa", updateCache = FALSE)

Perform pathway analysis using CePa ORA:

In [ ]:
# Specify the list of argument tailored for CePa ORA
CePaORAArgsList <- list(cen = "equal.weight", pThreshold = 0.05)

# Set seed to create reproducible results
set.seed(1)

# Run CePa ORA on Affymetrix dataset:
affyCePaORAResult <- RCPA::runPathwayAnalysis(summarizedExperiment = affyDEExperiment,
                                              network = CePaNetwork, method = "cepaORA", 
                                              CePaORAArgs = CePaORAArgsList)

# Run CePa ORA on Agilent dataset:
agilCePaORAResult <- RCPA::runPathwayAnalysis(summarizedExperiment = agilDEExperiment,
                                              network = CePaNetwork, method = "cepaORA", 
                                              CePaORAArgs = CePaORAArgsList)

# Run CePa ORA on RNA-Seq dataset:
RNASeqCePaORAResult <- RCPA::runPathwayAnalysis(summarizedExperiment = RNASeqDEExperiment, 
                                                network = CePaNetwork, method = "cepaORA", 
                                                CePaORAArgs = CePaORAArgsList)
                                                
# Display the result for RNA-Seq dataset (for example):
RNASeqCePaORAResult[1:5, c("ID", "p.value", "pFDR", "score", "name")]

Perform pathway analysis using CePa GSA:

In [ ]:
# Specify the list of arguments tailored for CePa GSA
CePaGSAArgsList <- list(cen = "equal.weight", nlevel = "tvalue_abs", plevel = "mean")

# Set seed to reproducible results
set.seed(1)

# Run CePa GSA for Affymetrix dataset
affyCePaGSAResult <- RCPA::runPathwayAnalysis(summarizedExperiment = affyDEExperiment,
                                              network = CePaNetwork, method = "cepaGSA", 
                                              CePaGSAArgs = CePaGSAArgsList)

# Run CePa GSA for Agilent dataset
agilCePaGSAResult <- RCPA::runPathwayAnalysis(summarizedExperiment = agilDEExperiment,
                                              network = CePaNetwork, method = "cepaGSA", 
                                              CePaGSAArgs = CePaGSAArgsList)

# Run CePa GSA on RNA-Seq dataset
RNASeqCePaGSAResult <- RCPA::runPathwayAnalysis(summarizedExperiment = RNASeqDEExperiment, 
                                                network = CePaNetwork, method = "cepaGSA", 
                                                CePaGSAArgs = CePaGSAArgsList)

# Display the result for RNA-Seq dataset (for example):
RNASeqCePaGSAResult[1:5, c("ID", "p.value", "pFDR", "score", "name")]

Visualize the results using volcano, forest plot and network of pathways:

In [ ]:
# Replace RNASeqSPIAResult with other results obtained from previous codes if needed
# Generate volcano plot for SPIA results
RCPA::plotVolcanoPathway(PAResult = RNASeqSPIAResult, topToLabel = 15) +
    ggplot2::ggtitle("RNASeq - GSE153873 - SPIA")

# Generate volcano plot for SPIA results
# Select top 20 pathways from the results
RNASeqSPIAToPlot <- list("RNASeq - GSE153873 - SPIA" = RNASeqSPIAResult[1:20,])
selectedPathways <- RNASeqSPIAResult$ID[1:20]
# Generate forest plot:
RCPA::plotForest(resultsList = RNASeqSPIAToPlot, yAxis = "name", statLims = c(-4, 8))

# Generate network graph of selected pathways
pltHtml <- RCPA::plotPathwayNetwork(
    PAResults = RNASeqSPIAToPlot,
    genesets = KEGGGenesets,
    selectedPathways = selectedPathways,
    statistic = "normalizedScore",
    mode = "continuous",
    edgeThreshold = 0.75
)

## **Basic Protocol 7: DATA INTEGRATION AND VISUALIZATION**

### **Gene-Level Meta-Analysis**

In [ ]:
# Install required dependencies beforehand
BiocManager::install("ggvenn", update = F)

# DE analysis result from Affymetrix dataset:
affyDEExperiment <- RCPA::loadData("affyDEExperiment")
# DE analysis result from Agilent dataset:
agilDEExperiment <- RCPA::loadData("agilDEExperiment")
# DE analysis result from RNA-Seq dataset:
RNASeqDEExperiment <- RCPA::loadData("RNASeqDEExperiment")

# Extract the differential analysis result obtained from previous protocols
affyDEResults <- SummarizedExperiment::rowData(affyDEExperiment)
agilDEResults <- SummarizedExperiment::rowData(agilDEExperiment)
RNASeqDEResults <- SummarizedExperiment::rowData(RNASeqDEExperiment)

# Prepare the input list of DE results
DEResults <- list(
    "Affymetrix - GSE5281" = affyDEResults,
    "Agilent - GSE61196" = agilDEResults,
    "RNASeq - GSE153873" = RNASeqDEResults
)

# Generate a venn diagram plot
RCPA::plotVennDE(DEResults = DEResults, topToList = 10)

In [ ]:

# Retrieve a list of common DE genes among multiple datasets
commonDEGenes <- RCPA::getCommonDEGenes(DEResults = DEResults)

# Display the commonDEGenes
commonDEGenes[1:5,]

In [ ]:
# Install required dependencies beforehand
BiocManager::install("meta", update = F)

# Perform meta-analysis using Stouffer’s method
metaDEResult <- RCPA::runDEMetaAnalysis(DEResults = DEResults, method = "stouffer")

# Display the result:
head(metaDEResult)

In [ ]:
# Select top 20 most significant genes:
genesToPlot <- metaDEResult$ID[1:20]

# Get the full description of the genesToPlot:
genesAnnotation <- RCPA::getEntrezAnnotation(genesToPlot)
labels <- genesAnnotation[genesToPlot, "Description"]

# Create a list containing the results from individual analysis and meta analysis
affyDEResults <- SummarizedExperiment::rowData(affyDEExperiment)
agilDEResults <- SummarizedExperiment::rowData(agilDEExperiment)
RNASeqDEResults <- SummarizedExperiment::rowData(RNASeqDEExperiment)

resultsToPlot <- list(
    "Affymetrix - GSE5281" = affyDEResults,
    "Agilent - GSE61196" = agilDEResults,
    "RNASeq - GSE153873" = RNASeqDEResults,
    "Meta-analysis" = metaDEResult
)
# Generate gene heatmap
plt <- RCPA::plotDEGeneHeatmap(DEResults = resultsToPlot, 
                        genes = genesToPlot, labels = labels, 
                        negLog10pValueLims = c(0, 5), logFCLims = c(-1, 1))

plt

In [ ]:
# Select columns in the results of differential analysis:
selectedColumns <- colnames(metaDEResult)
print(selectedColumns)

# Prepare the list for plotting
DEResultsToPlot <- list(
    "Affymetrix - GSE5281" = affyDEResults[, selectedColumns],
    "Agilent - GSE61196" = agilDEResults[, selectedColumns],
    "RNASeq - GSE153873" = RNASeqDEResults[, selectedColumns],
    "Meta-analysis" = metaDEResult
)
# Plot for KEGG Alzheimer’s Disease pathway
pltObj <- RCPA::plotKEGGMap(DEResults = DEResultsToPlot, 
                            KEGGPathwayID = "hsa05010", 
                            stat = "logFC", 
                            pThreshold = 1, statLimit = 1)

# This code is specific for Jupyter notebook environment to control the plot size
options(repr.plot.width = pltObj$width/250, repr.plot.height = pltObj$height/250)

# Display the plot
pltObj$plot

### **Pathway-Level Meta-Analysis**

In [ ]:
# FGSEA analysis result from Affymetrix dataset:
affyFGSEAResult <- RCPA::loadData("affyFGSEAResult")
# FGSEA analysis result from Agilent dataset:
agilFGSEAResult <- RCPA::loadData("agilFGSEAResult")
# FGSEA analysis result from RNA-Seq dataset:
RNASeqFGSEAResult <- RCPA::loadData("RNASeqFGSEAResult")

# # Load the KEGG gene sets
# KEGGGenesets <- RCPA::loadData("KEGGGenesets")

# Load the KEGG gene sets
KEGGGenesets <- RCPA::loadData("KEGGgenesets")

# Compile a list of pathway analysis results
PAResults <- list(
    "Affymetrix - GSE5281" = affyFGSEAResult,
    "Agilent - GSE61196" = agilFGSEAResult,
    "RNASeq - GSE153873" = RNASeqFGSEAResult
)

# This code is specific for Jupyter notebook environment to control the plot size
options(repr.plot.width = 10, repr.plot.height = 10)

# Plot venn diagram
RCPA::plotVennPathway(PAResults = PAResults, pThreshold = 0.05)

In [ ]:

# Query a list of common pathways
commonPathways <- RCPA::getCommonPathways(PAResults = PAResults)

# Display the results:
commonPathways[1:5,]

In [ ]:
# Meta-analysis using Stouffer’s method
metaPAResult <- RCPA::runPathwayMetaAnalysis(PAResults = PAResults, method = "stouffer")

# Display the results:
metaPAResult[1:5, 1:5]

In [ ]:
# Select top 30 significant from meta-analysis
selectedPathways <- metaPAResult$ID[1:30]
# Create a list of pathway analysis results of these 30 pathways
PAResultsToPlot <- list(
    "Affymetrix - GSE5281" = affyFGSEAResult,
    "Agilent - GSE61196" = agilFGSEAResult,
    "RNASeq - GSE153873" = RNASeqFGSEAResult,
    "Meta-analysis" = metaPAResult
)

# Plot bar chart
RCPA::plotBarChart(results = PAResultsToPlot, selectedPathways = selectedPathways) +
    ggplot2::ggtitle("FGSEA Analysis Results")

In [ ]:

# Plot heat map
RCPA::plotPathwayHeatmap(resultsList = PAResultsToPlot, 
                        yAxis = "name", 
                        selectedPathways = selectedPathways)

In [ ]:
# Select top 30 significant from meta-analysis
selectedPathways <- metaPAResult$ID[1:30]
# Create a list of pathway analysis results of these 30 pathways
allPAResultsToPlot <- list(
    "Affymetrix - GSE5281" = affyFGSEAResult,
    "Agilent - GSE61196" = agilFGSEAResult,
    "RNASeq - GSE153873" = RNASeqFGSEAResult,
    "Meta-analysis" = metaPAResult
)
# Plot pathway network
pltHtml <- RCPA::plotPathwayNetwork(
    PAResults = allPAResultsToPlot,
    genesets = KEGGGenesets,
    selectedPathways = selectedPathways,
    statistic = "normalizedScore",
    mode = "continuous",
    edgeThreshold = 0.75
)


### **Pathway-Level Consensus Analysis**

In [ ]:
# Wilcoxon test results for RNA-Seq dataset:
RNASeqWilcoxResult <- RCPA::loadData("RNASeqWilcoxResult")
# FGSEA results for RNA-Seq dataset:
RNASeqFGSEAResult <- RCPA::loadData("RNASeqFGSEAResult")
# SPIA results for RNA-Seq dataset:
RNASeqSPIAResult <- RCPA::loadData("RNASeqSPIAResult")
# Load the KEGG gene sets
KEGGGenesets <- RCPA::loadData("KEGGGenesets")

# Prepare a list of results obtained from Wilcox test, FGSEA, and SPIA
selectedRNASeqPAResults <- list(
    "Wilcox" = RNASeqWilcoxResult,
    "FGSEA" = RNASeqFGSEAResult,
    "SPIA" = RNASeqSPIAResult
)

# Plot Venn diagram
RCPA::plotVennPathway(PAResults= selectedRNASeqPAResults, pThreshold = 0.05)

In [ ]:
# Query a list of common pathways from Wilcox Test, FGSEA, SPIA:
commonPathways <- RCPA::getCommonPathways(PAResults = selectedRNASeqPAResults)
# Display the result
commonPathways[1:5,]

In [ ]:
# Set seed to create reproducible result:
set.seed(1)

# Run consensus analysis using weightedZMean:
consensusWZRNASeqPAResult <- RCPA::runConsensusAnalysis(PAResults = selectedRNASeqPAResults, 
                                                        method = "weightedZMean")

# Display the result
consensusWZRNASeqPAResult[1:6, c("ID", "p.value", "pFDR", "name")]

# Run consensus analysis using RRA
consensusRRARNASeqPAResult <- RCPA::runConsensusAnalysis(PAResults = selectedRNASeqPAResults, 
                                                        method = "RRA", rank.by = "both")

# Display the result
consensusRRARNASeqPAResult[1:5, c("ID", "p.value", "pFDR", "name")]

In [ ]:
# Generate pathway network for consensus analysis:
# Create a list of results to plot:
selectedCCRNASeqPAResults <- list(
    "Wilcox" = RNASeqWilcoxResult,
    "FGSEA" = RNASeqFGSEAResult,
    "SPIA" = RNASeqSPIAResult,
    "Consensus - weightedZMean" = consensusWZRNASeqPAResult
)

# Select to plot top 30 most significant pathways
# from consensus analysis using weightedZMean:
selectedPathways <- consensusWZRNASeqPAResult$ID[1:30]

# Plot pathway network
pltHtml <- RCPA::plotPathwayNetwork(
    PAResults = selectedCCRNASeqPAResults,
    genesets = KEGGGenesets,
    selectedPathways = selectedPathways,
    statistic = "p.value",
    mode = "discrete",
    pThreshold = 0.05,
    edgeThreshold = 0.75
)

***Perform consensus analysis of multiple methods and datasets:***

In [ ]:
# Load the necessary data if users skipped the previous protocols
# Enrichment results using Wilcoxon test:
affyWilcoxResult <- RCPA::loadData("affyWilcoxResult")
RNASeqWilcoxResult <- RCPA::loadData("RNASeqWilcoxResult")
agilWilcoxResult <- RCPA::loadData("agilWilcoxResult")

# Enrichment results FGSEA:
affyFGSEAResult <- RCPA::loadData("affyFGSEAResult")
agilFGSEAResult <- RCPA::loadData("agilFGSEAResult")
RNASeqFGSEAResult <- RCPA::loadData("RNASeqFGSEAResult")

# TB analysis using SPIA:
affySPIAResult <- RCPA::loadData("affySPIAResult")
agilSPIAResult <- RCPA::loadData("agilSPIAResult")
RNASeqSPIAResult <- RCPA::loadData("RNASeqSPIAResult")

# Load the KEGG gene sets
KEGGGenesets <- RCPA::loadData("KEGGGenesets")

# Create a list of results from multiple pathway analysis methods on multiple datasets:
selectedPAResults <- list(
    "Affymetrix - Wilcox" = affyWilcoxResult,
    "Affymetrix - FGSEA" = affyFGSEAResult,
    "Affymetrix - SPIA" = affySPIAResult,
    "Agilent - Wilcox" = agilWilcoxResult,
    "Agilent - FGSEA" = agilFGSEAResult,
    "Agilent - SPIA" = agilSPIAResult,
    "RNASeq - Wilcox" = RNASeqWilcoxResult,
    "RNASeq - FGSEA" = RNASeqFGSEAResult,
    "RNASeq - SPIA" = RNASeqSPIAResult
)
# Run consensus analysis using weightedZMean on selectedPAResults:
consensusPAResult <- RCPA::runConsensusAnalysis(PAResults = selectedPAResults, 
                                                method = "weightedZMean")
                                                
# display the results
consensusPAResult[1:6, c("ID", "p.value", "pFDR", "name")]

In [ ]:
# Select top 20 significant from consensus analysis
selectedPathways <- consensusPAResult$ID[1:20]
# Create a list of results to plot:
selectedPAResultsToPlot <- list(
    "Affymetrix - Wilcox" = affyWilcoxResult,
    "Affymetrix - FGSEA" = affyFGSEAResult,
    "Affymetrix - SPIA" = affySPIAResult,
    "Agilent - Wilcox" = agilWilcoxResult,
    "Agilent - FGSEA" = agilFGSEAResult,
    "Agilent - SPIA" = agilSPIAResult,
    "RNASeq - Wilcox" = RNASeqWilcoxResult,
    "RNASeq - FGSEA" = RNASeqFGSEAResult,
    "RNASeq - SPIA" = RNASeqSPIAResult,
    "Consensus Analaysis" = consensusPAResult
)
# Plot pathway network
pltHtml <- RCPA::plotPathwayNetwork(
    PAResults = selectedPAResultsToPlot,
    genesets = KEGGGenesets,
    selectedPathways = selectedPathways,
    statistic = "p.value",
    mode = "discrete",
    edgeThreshold = 0.75
)

In [ ]:
# This code is specific for Jupyter notebook environment to control the plot size
options(repr.plot.width = 20, repr.plot.height = 10)

RCPA::plotPathwayHeatmap(resultsList = selectedPAResultsToPlot, 
                        yAxis = "name", 
                        selectedPathways = selectedPathways)